In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

np.random.seed(130298) # to have the same random numbers

In [2]:
%pwd

'/Users/annachiararossi/Documents/Healthcare data/heart-failure-project'

In [3]:
drugs = pd.read_csv('dat_md.csv')
drugs.drop(columns = ['Unnamed: 0'], inplace = True)
drugs.shape

(12654, 2)

We notice that this dataset has much more rows than the other: so we want to check if there are patients for which there is more than one row in the table (i.e. patients that take more than one drug). There are indeed a lot of them (10650).

In [4]:
drugs.head()

,inpatient.number,Drug_name
0,857781,sulfotanshinone sodium injection
1,857781,Furosemide tablet
2,857781,Meglumine Adenosine Cyclophosphate for injection
3,857781,Furosemide injection
4,857781,Milrinone injection


We observe the different drugs administrated to the patients.

In [5]:
drugs['Drug_name'].value_counts()

Spironolactone tablet                               1833
Furosemide injection                                1718
Furosemide tablet                                   1641
Meglumine Adenosine Cyclophosphate for injection    1114
Deslanoside injection                               1015
Digoxin tablet                                       998
Atorvastatin calcium tablet                          822
Milrinone injection                                  707
sulfotanshinone sodium injection                     570
Benazepril hydrochloride tablet                      434
Valsartan Dispersible tablet                         348
Shenfu injection                                     338
Isosorbide Mononitrate Sustained Release tablet      326
Hydrochlorothiazide tablet                           283
Torasemide tablet                                    252
Nitroglycerin injection                              203
Isoprenaline Hydrochloride injection                  30
Dobutamine hydrochloride inject

First, we unite into only one drug the Furosemide given by injection and by tablet, since it is indeed the same drug, only subministrated in different ways.

In [6]:
new_col = drugs['Drug_name'].copy()
new_col[new_col == 'Furosemide injection'] = 'Furosemide'
new_col[new_col == 'Furosemide tablet'] = 'Furosemide'
drugs['Drug_name'] = new_col

Now we remove the rows corresponding to the two drugs taken only by a very small number of persons, since we will not be able to infer something with so few patients taking them.

In [7]:
too_few1 = list(drugs.index[drugs['Drug_name'] == 'Isoprenaline Hydrochloride injection'])
too_few2 = list(drugs.index[drugs['Drug_name'] == 'Dobutamine hydrochloride injection'])

def union(lst1, lst2):
    final_list = list(set(lst1) | set(lst2))
    return final_list

too_few = union(too_few1, too_few2)
drugs.drop(too_few, inplace = True)
drugs.reset_index(inplace=True, drop=True)

In [8]:
sum(drugs.duplicated())

1446

Checking again for duplicated rows, we notice that now there are 1446 repeated rows (these will correspond to the patients taking Furosemide both by injection and by tablet). So we remove these duplicates (and we obtain a dataset with 11156 rows).

In [9]:
drugs.drop_duplicates(inplace=True)
drugs.reset_index(inplace=True, drop=True)
drugs

,inpatient.number,Drug_name
0,857781,sulfotanshinone sodium injection
1,857781,Furosemide
2,857781,Meglumine Adenosine Cyclophosphate for injection
3,857781,Milrinone injection
4,857781,Deslanoside injection
...,...,...
11151,791864,Spironolactone tablet
11152,791864,Valsartan Dispersible tablet
11153,791864,Digoxin tablet
11154,791864,Deslanoside injection


In [10]:
sum(drugs["inpatient.number"].duplicated())

9152

However we are still far from the number of patients (2008). This means that many patients will take more than one medicine.

# Grouping drugs according to their aim

We try to group the different drugs according to their aim and/or their acting principle.
From literature, we found that these drugs can be subdivided into 5 main groups:
- diuretics: 'Furosemide', 'Spironolactone tablet', 'Hydrochlorothiazide tablet', 'Torasemide tablet'
- anti-hypertension: 'Spironolactone tablet', 'Benazepril hydrochloride tablet', 'Valsartan Dispersible tablet'
- heart failure: 'Meglumine Adenosine Cyclophosphate for injection', 'Deslanoside injection', 'Shenfu injection', 'Nitroglycerin injection'
- angina and other cardiac problems: 'Meglumine Adenosine Cyclophosphate for injection', 'Digoxin tablet', 'Milrinone injection', 'sulfotanshinone sodium injection', 'Benazepril hydrochloride tablet', 'Valsartan Dispersible tablet', 'Isosorbide Mononitrate Sustained Release tablet', 'Nitroglycerin injection'
- lowering lipids: 'Atorvastatin calcium tablet'

In [11]:
drugs_by_aim=drugs.copy()

In [12]:
DIURETICS= ['Furosemide', 'Spironolactone tablet', 'Hydrochlorothiazide tablet', 'Torasemide tablet']
HYPERTENSION = ['Spironolactone tablet', 'Benazepril hydrochloride tablet', 'Valsartan Dispersible tablet']
HEART_FAILURE = ['Meglumine Adenosine Cyclophosphate for injection', 'Deslanoside injection', 'Shenfu injection',
                 'Nitroglycerin injection']
ANGINA_ETAL = ['Meglumine Adenosine Cyclophosphate for injection', 'Digoxin tablet', 'Milrinone injection',
               'sulfotanshinone sodium injection', 'Benazepril hydrochloride tablet', 
               'Valsartan Dispersible tablet', 'Isosorbide Mononitrate Sustained Release tablet', 
               'Nitroglycerin injection']
CHOLESTEROL = ['Atorvastatin calcium tablet']

# create a list containing these groups of drugs

list_of_groups = [DIURETICS, HYPERTENSION, HEART_FAILURE, ANGINA_ETAL, CHOLESTEROL]

In [13]:
drug_group = ['diuretics','hypertension','heart_failure','angina_etal','cholesterol']

for group_idx,drug in enumerate(drug_group):
    
    new_col = drugs_by_aim['Drug_name'].copy()
    
    for i in range(drugs.shape[0]):
        if new_col[i] in list_of_groups[group_idx]:
            new_col[i]=1
        else:
            new_col[i]=0
            
    drugs_by_aim[drug]= new_col

drugs_by_aim.drop('Drug_name', axis=1, inplace=True)


We also drop the column where we specify the particular name of the drugs and we only keep into account their aim.
At this point we check again for duplicated rows (which will be the patients taking more than one drug of the same group) and we drop them.

In [14]:
drugs_by_aim[drugs_by_aim['inpatient.number'] == 857781]

,inpatient.number,diuretics,hypertension,heart_failure,angina_etal,cholesterol
0,857781,0,0,0,1,0
1,857781,1,0,0,0,0
2,857781,0,0,1,1,0
3,857781,0,0,0,1,0
4,857781,0,0,1,0,0
5,857781,1,0,0,0,0
6,857781,0,1,0,1,0
7,857781,0,0,0,0,1
8,857781,0,0,0,1,0


We don't need to drop duplicates, but we need to get from each patient ID a row having 1 if the patient has at least a 1 in that column, 0 otherwise. This can simply be achieved by taking the max value for each patient on each column.

In [15]:
df_grouped = drugs_by_aim.groupby(['inpatient.number'])
df_grouped.ngroups

2004

In [16]:
drugs_by_patient = df_grouped.max()

In [17]:
drugs_by_patient.head()

,diuretics,hypertension,heart_failure,angina_etal,cholesterol
inpatient.number,,,,,
722128,1,1,1,1,0
723327,1,1,1,1,1
723617,1,1,0,0,1
724385,1,1,1,1,0
725509,1,1,1,1,0


Now we want to merge these two datasets over the patient ID (based on the patient ID in the main dataframe, because some patients have already been removed for other reasons).

In [18]:
data = pd.read_csv('data_cleaned.csv')
data.set_index('inpatient.number', inplace = True)
data.head()

,body.temperature,pulse,respiration,systolic.blood.pressure,diastolic.blood.pressure,weight,BMI,fio2,left.ventricular.end.diastolic.diameter.LV,creatinine.enzymatic.method,...,consciousness_ResponsiveToPain,consciousness_ResponsiveToSound,"ageCat_(21,29]","ageCat_(29,39]","ageCat_(39,49]","ageCat_(49,59]","ageCat_(59,69]","ageCat_(69,79]","ageCat_(79,89]","ageCat_(89,110]"
inpatient.number,,,,,,,,,,,,,,,,,,,,,
857781,36.7,87.0,19.0,102.0,64.0,50.0,18.590125,33,NaN,108.3,...,0,0,0,0,0,0,0,1,0,0
743087,36.8,95.0,18.0,150.0,70.0,51.0,19.195303,33,40.0,62.0,...,0,0,0,0,0,0,0,1,0,0
866418,36.5,98.0,18.0,102.0,67.0,70.0,24.221453,33,46.0,185.1,...,0,0,0,0,0,0,1,0,0,0
775928,36.0,73.0,19.0,110.0,74.0,65.0,22.491349,33,NaN,104.8,...,0,0,0,0,0,0,0,1,0,0
810128,35.0,88.0,19.0,134.0,62.0,76.0,31.633715,33,64.0,83.9,...,0,0,0,0,0,0,0,1,0,0


In [19]:
# suffix is needed because we already have a variable called cholesterol
data_final = data.join(drugs_by_patient, rsuffix = '_drug')
print(data.shape,data_final.shape)
data_final.head()

(1970, 105) (1970, 110)


,body.temperature,pulse,respiration,systolic.blood.pressure,diastolic.blood.pressure,weight,BMI,fio2,left.ventricular.end.diastolic.diameter.LV,creatinine.enzymatic.method,...,"ageCat_(49,59]","ageCat_(59,69]","ageCat_(69,79]","ageCat_(79,89]","ageCat_(89,110]",diuretics,hypertension,heart_failure,angina_etal,cholesterol_drug
inpatient.number,,,,,,,,,,,,,,,,,,,,,
857781,36.7,87.0,19.0,102.0,64.0,50.0,18.590125,33,NaN,108.3,...,0,0,1,0,0,1.0,1.0,1.0,1.0,1.0
743087,36.8,95.0,18.0,150.0,70.0,51.0,19.195303,33,40.0,62.0,...,0,0,1,0,0,1.0,0.0,0.0,0.0,0.0
866418,36.5,98.0,18.0,102.0,67.0,70.0,24.221453,33,46.0,185.1,...,0,1,0,0,0,1.0,1.0,1.0,1.0,0.0
775928,36.0,73.0,19.0,110.0,74.0,65.0,22.491349,33,NaN,104.8,...,0,0,1,0,0,1.0,1.0,1.0,1.0,1.0
810128,35.0,88.0,19.0,134.0,62.0,76.0,31.633715,33,64.0,83.9,...,0,0,1,0,0,1.0,1.0,1.0,1.0,0.0


In [20]:
# convert the last five columns to int again
data_final.iloc[:,-5:] = data_final.iloc[:,-5:].astype('Int64')
data_final.head()

,body.temperature,pulse,respiration,systolic.blood.pressure,diastolic.blood.pressure,weight,BMI,fio2,left.ventricular.end.diastolic.diameter.LV,creatinine.enzymatic.method,...,"ageCat_(49,59]","ageCat_(59,69]","ageCat_(69,79]","ageCat_(79,89]","ageCat_(89,110]",diuretics,hypertension,heart_failure,angina_etal,cholesterol_drug
inpatient.number,,,,,,,,,,,,,,,,,,,,,
857781,36.7,87.0,19.0,102.0,64.0,50.0,18.590125,33,NaN,108.3,...,0,0,1,0,0,1,1,1,1,1
743087,36.8,95.0,18.0,150.0,70.0,51.0,19.195303,33,40.0,62.0,...,0,0,1,0,0,1,0,0,0,0
866418,36.5,98.0,18.0,102.0,67.0,70.0,24.221453,33,46.0,185.1,...,0,1,0,0,0,1,1,1,1,0
775928,36.0,73.0,19.0,110.0,74.0,65.0,22.491349,33,NaN,104.8,...,0,0,1,0,0,1,1,1,1,1
810128,35.0,88.0,19.0,134.0,62.0,76.0,31.633715,33,64.0,83.9,...,0,0,1,0,0,1,1,1,1,0


In [35]:
data_final.iloc[:,-5:].sum()

diuretics           1939
hypertension        1827
heart_failure       1612
angina_etal         1845
cholesterol_drug     807
dtype: int64

In [23]:
data_final.shape[0]

1970

In [51]:
for i in range(6):
    print(f'Number of patients taking {i} drugs: {len(np.where(data_final.iloc[:,-5:].sum(axis=1)==i)[0])}')

Number of patients taking 0 drugs: 4
Number of patients taking 1 drugs: 12
Number of patients taking 2 drugs: 73
Number of patients taking 3 drugs: 236
Number of patients taking 4 drugs: 1061
Number of patients taking 5 drugs: 584


In [ ]:
#data_final.to_csv('merged_data.csv')